In [ ]:
!pip install onnx
!pip install medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=0fb31abc5b69e1b30ff7d9eaac28aa3a9b6ecfea5f6d39ec4b136cc1b0fb5a1b
  Stored in directory: /root/.cache/pip/wheels/90/d4/f7/9404e5db0116bd4d43e5666eaa3e70ab53723e1e3ea40c9a95
Successfully built fire


In [ ]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Insert the directory
import sys
sys.path.insert(0,'/content/drive/MyDrive/Colab Notebooks/domain_adaptation')

Mounted at /content/drive


In [ ]:
import itertools
import random
import time

import numpy as np
import torch
import onnx
from PIL import Image
from torchvision.models import ResNet50_Weights

from medmnist_data import MedMnistData
from model_trainer import ModelTrainer

In [8]:
def cont_learning(base_model, dataset_names, dropout, ewc_lambda, gamma,
                learning_rate, momentum, num_classes, num_epochs,
                preprocess_resnet, rehearsal, step_size, trim_test_dataset,
                trim_train_dataset):
    datasets = []
    smaller_size = 1e6
    smaller_test_size = 0
    for name in dataset_names:
        dataset = MedMnistData(name.lower())
        dataset.select_n_classes(num_classes)
        dataset_size = len(dataset.train_dataset.imgs)

        if dataset_size < smaller_size:
            smaller_size = dataset_size
            smaller_test_size = len(dataset.test_dataset.imgs)

        if preprocess_resnet:
            weights = ResNet50_Weights.DEFAULT
            preprocess = weights.transforms()
            imgs = dataset.train_dataset.imgs
            n_samples = len(imgs)

            # Create an empty tensor to hold the transformed images
            transformed_images = torch.empty(n_samples, 3, 224, 224)

            for i in range(n_samples):
                img = Image.fromarray(imgs[i])
                img = preprocess(img)
                transformed_images[i] = img

            dataset.train_dataset.imgs = transformed_images

        datasets.append(dataset)

    datasets = sorted(datasets, key=lambda dataset: len(
        dataset.train_dataset.imgs), reverse=False)
    random_index = random.randint(0, len(datasets) - 1)
    # Pop the dataset at the random index
    random_dataset = datasets.pop(random_index)
    # Append the random dataset to the end of the list
    datasets.append(random_dataset)
    dataset_names = []

    for dataset in datasets:
        dataset_names.append(dataset.test_dataset.flag)
    print(dataset_names)

    for dataset in datasets:
        if trim_train_dataset:
            imgs = dataset.train_dataset.imgs
            labels = dataset.train_dataset.labels
            n_samples = len(imgs)
            # Randomly sample a subset of the data
            random_idx = np.random.choice(n_samples, smaller_size,
                                          replace=False)
            # Concatenate the subset to your dataset
            dataset.train_dataset.imgs = imgs[random_idx]
            dataset.train_dataset.labels = labels[random_idx]

        if trim_test_dataset:
            imgs = dataset.test_dataset.imgs
            labels = dataset.test_dataset.labels
            n_samples = len(imgs)
            # Randomly sample a subset of the data
            random_idx = np.random.choice(n_samples, smaller_test_size,
                                          replace=False)
            # Concatenate the subset to your dataset
            dataset.test_dataset.imgs = imgs[random_idx]
            dataset.test_dataset.labels = labels[random_idx]
    # Initialize the class
    model_trainer = ModelTrainer(num_classes=num_classes,
                                  model_name=base_model,
                                  learning_rate=learning_rate,
                                  momentum=momentum,
                                  gamma=gamma,
                                  step_size=step_size,
                                  dropout=dropout)
    # Train on the datasets specified parameters
    accuracy, f1_macro, f1_weighted, ari, nmi = \
        model_trainer.train_on_datasets(datasets=datasets[:-1],
                                        num_epochs=num_epochs,
                                        eval_dataset=datasets[-1],
                                        rehearsal=rehearsal,
                                        ewc_lambda=ewc_lambda)
    timestr = time.strftime("%Y%m%d-%H%M%S")
    dir = '/content/drive/MyDrive/Colab Notebooks/domain_adaptation/models/'
    model_name = dir + 'medmnist_model_' + timestr + '.onnx'
    model_trainer.save_model(model_name, 'onnx')
    report = f"""\
    {timestr}
    # Model parameters
    num_classes = {num_classes}
    base_model = '{base_model}'
    preprocess_resnet = {preprocess_resnet}

    # Optimizer parameters
    learning_rate = {learning_rate}
    momentum = {momentum}

    # Scheduler parameters
    gamma = {gamma}
    step_size = {step_size}

    # Training parameters
    num_epochs = {num_epochs}
    trim_train_dataset = {trim_train_dataset}
    trim_test_dataset = {trim_test_dataset}

    # Regularization parameters
    ewc_lambda = {ewc_lambda}
    rehearsal = {rehearsal}
    dropout = {dropout}

    # Datasets order
    dataset_names = {dataset_names}

    # Model metrics
    accuracy = {accuracy}
    f1_macro = {f1_macro}
    f1_weighted = {f1_weighted}
    ari = {ari}
    nmi = {nmi}

    # Output model name
    model_name = '{model_name}'

    --------------------------------------------------------------------
    """
    with open('/content/drive/MyDrive/Colab Notebooks/domain_adaptation/training_report6.txt', 'a+') as f:
        f.write(report)

In [ ]:
# Model parameters
num_classes = 6
base_model = 'resnet_50_finetune'
preprocess_resnet = False
# Optimizer parameters
learning_rate_values = [0.001]
momentum = 0.9
# Scheduler parameters
gamma_values = [0.9, 1.0]
step_size_values = [10]
# Training parameters
num_epochs_values = [10, 30]
trim_train_dataset = False
trim_test_dataset = False
# Regularization parameters
ewc_lambda_values = [0, 0.001]
rehearsal_values = [0.1, 0, 0.05]
dropout_values = [0.5, 0.4, 0.6]

# Load and preprocess your datasets
dataset_names = ['PathMNIST', 'DermaMNIST', 'BloodMNIST', 'TissueMNIST',
                  'OrganCMNIST', 'OrganAMNIST', 'OrganSMNIST']
random.Random().shuffle(dataset_names)

# Create a list of parameter lists
parameter_lists = [num_epochs_values, ewc_lambda_values, step_size_values,
                   gamma_values, learning_rate_values, rehearsal_values,
                   dropout_values]

# Iterate over all combinations
for combination in itertools.product(*parameter_lists):
    num_epochs, ewc_lambda, step_size, gamma, learning_rate, rehearsal, dropout = combination
    cont_learning(base_model, dataset_names, dropout, ewc_lambda, gamma,
                learning_rate, momentum, num_classes, num_epochs,
                preprocess_resnet, rehearsal, step_size, trim_test_dataset,
                trim_train_dataset)

Using downloaded and verified file: /root/.medmnist/organamnist.npz
Using downloaded and verified file: /root/.medmnist/organamnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/dermamnist.npz
Using downloaded and verified file: /root/.medmnist/dermamnist.npz
Using downloaded and verified file: /root/.medmnist/organcmnist.npz
Using downloaded and verified file: /root/.medmnist/organcmnist.npz
Using downloaded and verified file: /root/.medmnist/organsmnist.npz
Using downloaded and verified file: /root/.medmnist/organsmnist.npz
Using downloaded and verified file: /root/.medmnist/bloodmnist.npz
Using downloaded and verified file: /root/.medmnist/bloodmnist.npz
Using downloaded and verified file: /root/.medmnist/tissuemnist.npz
Using downloaded and verified file: /root/.medmnist/tissuemnist.npz
['organcmnist', 'bloodmnist', 'organsmnist', 'organamnis

100%|██████████| 62/62 [00:02<00:00, 28.07it/s]


Accuracy on dermamnist: 719/1982 = 0.3627648839556004
Accuracy sklearn on dermamnist: 0.3627648839556004
F1 (macro) score on dermamnist: 0.16274726657662814
F1 (weighted) score on dermamnist: 0.4184610678008508
ARI on dermamnist: 0.04095853636353299
NMI on dermamnist: 0.024165203084546276
Average loss 1.581396271073373

[0] train on organcmnist
train Loss: 1.6133 Loss crit: 1.6133 Loss ewc: 0.0000 Acc: 0.3650
train Loss: 1.4278 Loss crit: 1.4278 Loss ewc: 0.0000 Acc: 0.4706
train Loss: 1.3897 Loss crit: 1.3897 Loss ewc: 0.0000 Acc: 0.4864
train Loss: 1.3607 Loss crit: 1.3607 Loss ewc: 0.0000 Acc: 0.5012
train Loss: 1.3190 Loss crit: 1.3190 Loss ewc: 0.0000 Acc: 0.5142
train Loss: 1.3228 Loss crit: 1.3228 Loss ewc: 0.0000 Acc: 0.5048
train Loss: 1.3109 Loss crit: 1.3109 Loss ewc: 0.0000 Acc: 0.5108
train Loss: 1.3037 Loss crit: 1.3037 Loss ewc: 0.0000 Acc: 0.5116
train Loss: 1.2875 Loss crit: 1.2875 Loss ewc: 0.0000 Acc: 0.5187
train Loss: 1.2876 Loss crit: 1.2876 Loss ewc: 0.0000 Acc: 

100%|██████████| 183/183 [00:03<00:00, 50.06it/s]


Accuracy on organcmnist: 3337/5856 = 0.5698428961748634
Accuracy sklearn on organcmnist: 0.5698428961748634
F1 (macro) score on organcmnist: 0.5233563217642121
F1 (weighted) score on organcmnist: 0.5559509763751754
ARI on organcmnist: 0.27657021593920367
NMI on organcmnist: 0.23467968958095292
Average loss 1.6378236181749022


...

Accuracy on dermamnist: 499/1982 = 0.25176589303733604
Accuracy sklearn on dermamnist: 0.25176589303733604
F1 (macro) score on dermamnist: 0.11815352728109613
F1 (weighted) score on dermamnist: 0.3163329403598362
ARI on dermamnist: -0.014989292426245294
NMI on dermamnist: 0.012975525981090546
Average loss 2.182290589653038

[4] train on pathmnist
imgs before (64738, 28, 28, 3), labels before (64738, 1)
img subset (6473, 28, 28, 3), labels subset (6473, 1)
imgs (71211, 28, 28, 3), labels (71211, 1)
imgs before (71211, 28, 28, 3), labels before (71211, 1)
img subset (6473, 28, 28, 3), labels subset (6473, 1)
imgs (77684, 28, 28, 3), labels (77684, 1)
imgs before (77684, 28, 28, 3), labels before (77684, 1)
img subset (6473, 28, 28, 3), labels subset (6473, 1)
imgs (84157, 28, 28, 3), labels (84157, 1)
imgs before (84157, 28, 28, 3), labels before (84157, 1)
img subset (6473, 28, 28, 3), labels subset (6473, 1)
imgs (90630, 28, 28, 3), labels (90630, 1)
train Loss: 1.5111 Loss crit: 1.511

100%|██████████| 127/127 [00:02<00:00, 53.78it/s]


Accuracy on pathmnist: 2974/4064 = 0.7317913385826772
Accuracy sklearn on pathmnist: 0.7317913385826772
F1 (macro) score on pathmnist: 0.6615187609625961
F1 (weighted) score on pathmnist: 0.7312095265507943
ARI on pathmnist: 0.5650921779485816
NMI on pathmnist: 0.5078511805583269
Average loss 0.7802458996263796


100%|██████████| 62/62 [00:01<00:00, 40.64it/s]


Accuracy on dermamnist: 81/1982 = 0.040867810292633706
Accuracy sklearn on dermamnist: 0.040867810292633706
F1 (macro) score on dermamnist: 0.04178787213951863
F1 (weighted) score on dermamnist: 0.03636330691014167
ARI on dermamnist: -0.0037210607076313177
NMI on dermamnist: 0.024902687474257407
Average loss 3.2038729106622075

[5] train on tissuemnist
imgs before (151895, 28, 28, 3), labels before (151895, 1)
img subset (9142, 28, 28, 3), labels subset (9142, 1)
imgs (161037, 28, 28, 3), labels (161037, 1)
imgs before (161037, 28, 28, 3), labels before (161037, 1)
img subset (10258, 28, 28, 3), labels subset (10258, 1)
imgs (171295, 28, 28, 3), labels (171295, 1)
imgs before (171295, 28, 28, 3), labels before (171295, 1)
img subset (10423, 28, 28, 3), labels subset (10423, 1)
imgs (181718, 28, 28, 3), labels (181718, 1)
imgs before (181718, 28, 28, 3), labels before (181718, 1)
img subset (15189, 28, 28, 3), labels subset (15189, 1)
imgs (196907, 28, 28, 3), labels (196907, 1)
imgs be